In [ ]:
# For obtaining FQ and RU
import os
import json
import glob
import math

# standard case: when the retrain values are unknown
def unlearning_vs_retrain(                              
    Unlearning_record,
    Retrain_record
):
    """
    1) Input:
       - Unlearning_record: A large list of length N, each element is a sublist of length 6
       - Retrain_record:    A large list of length N, each element is a sublist of length 6
         Note: N is the same for both

    2) Objective:
       - For each sublist in Unlearning_record:
         * Take the product of the first 3 values, then compute the cubic root => Unlearning_FQ_list
         * Take the product of the last 3 values, then compute the cubic root => Unlearning_RU_list
       - For each sublist in Retrain_record:
         * Take the product of the first 3 values, then compute the cubic root => Retrain_FQ_list
         * Take the product of the last 3 values, then compute the cubic root => Retrain_RU_list

       - Comparison:
         For the same index i:
           A = Unlearning_FQ_list[i]
           B = Retrain_FQ_list[i]
           If A < B,          result = A / B
           If B < A < 2B,     result = 2 - A / B
           Else               result = 0
         => This produces FQ_list

         Similarly, compare Unlearning_RU_list[i] and Retrain_RU_list[i] => RU_list

    3) Return:
       FQ_list, RU_list (both of size N)
    """

    # Function to compute cubic root, can also use (x**(1/3)) directly
    def cubic_root(x):
        # Since the product may be < 0 or floating point errors, handle with abs and preserve the sign
        # If all elements >= 0, simply return x**(1/3)
        sign = 1 if x >= 0 else -1
        return sign * (abs(x) ** (1/3))

    # Step 1: Construct four lists
    Unlearning_FQ_list = []
    Unlearning_RU_list = []
    Retrain_FQ_list = []
    Retrain_RU_list = []

    # Assume the two large lists have the same length
    N = len(Unlearning_record)
    for i in range(N):
        # Extract each sublist
        u_item = Unlearning_record[i]
        r_item = Retrain_record[i]

        # Compute product of first three and last three values => cubic root
        # --- Unlearning ---
        u_fq_val = cubic_root(u_item[0] * u_item[1] * u_item[2])*100
        u_ru_val = cubic_root(u_item[3] * u_item[4] * u_item[5])*100
        Unlearning_FQ_list.append(u_fq_val)
        Unlearning_RU_list.append(u_ru_val)

        # --- Retrain ---
        r_fq_val = cubic_root(r_item[0] * r_item[1] * r_item[2])*100
        r_ru_val = cubic_root(r_item[3] * r_item[4] * r_item[5])*100
        Retrain_FQ_list.append(r_fq_val)
        Retrain_RU_list.append(r_ru_val)

    # Step 2: Comparison rules
    # A = Unlearning_FQ_list[i], B = Retrain_FQ_list[i] => FQ_list
    # A = Unlearning_RU_list[i], B = Retrain_RU_list[i] => RU_list

    def compare_vals(A_list, B_list):
        """
        Return the comparison list, with the same length as A_list and B_list.
        If A < B,        => A/B
        If B < A < 2B,   => 2 - A/B
        Else             => 0
        """
        output = []
        for A, B in zip(A_list, B_list):
            if B == 0:
                # If B=0, handle to avoid division by zero. Here we simply set it to 0.
                output.append(0)
            else:
                if A < B:
                    output.append(A / B)
                elif B < A < 2 * B:
                    output.append(2 - (A / B))
                else:
                    output.append(0)
        return output

    # Step 3: Construct final results FQ_list, RU_list
    FQ_list = compare_vals(Unlearning_FQ_list, Retrain_FQ_list)
    RU_list = compare_vals(Unlearning_RU_list, Retrain_RU_list)

    # Return the results
    return FQ_list, RU_list

# gold_standard case: when the retrain values are already known
def FQ_RU(                              
    Unlearning_record,
    Retrain_FQ_list,
    Retrain_RU_list
):
    """
    1) Input:
       - Unlearning_record: A large list of length N, each element is a sublist of length 6
       - Retrain_FQ_list, Retrain_RU_list: Pre-computed gold standard lists
         Note: N is consistent

    2) Objective:
       - For each sublist in Unlearning_record:
         * Take the product of the first 3 values, then compute the cubic root => Unlearning_FQ_list
         * Take the product of the last 3 values, then compute the cubic root => Unlearning_RU_list

       - Comparison:
         For the same index i:
           A = Unlearning_FQ_list[i]
           B = Retrain_FQ_list[i]
           If A < B,          result = A / B
           If B < A < 2B,     result = 2 - A / B
           Else               result = 0
         => This produces FQ_list

         Similarly, compare Unlearning_RU_list[i] and Retrain_RU_list[i] => RU_list

    3) Return:
       FQ_list, RU_list (both of size N)
    """

    # Function to compute cubic root, can also use (x**(1/3)) directly
    def cubic_root(x):
        # Since the product may be < 0 or floating point errors, handle with abs and preserve the sign
        # If all elements >= 0, simply return x**(1/3)
        sign = 1 if x >= 0 else -1
        return sign * (abs(x) ** (1/3))

    # Step 1: Construct two lists
    Unlearning_FQ_list = []
    Unlearning_RU_list = []

    # Assume the two large lists have the same length
    N = len(Unlearning_record)
    for i in range(N):
        # Extract each sublist
        u_item = Unlearning_record[i]

        # Compute product of first three and last three values => cubic root
        # --- Unlearning ---
        u_fq_val = cubic_root(u_item[0] * u_item[1] * u_item[2])*100
        u_ru_val = cubic_root(u_item[3] * u_item[4] * u_item[5])*100
        Unlearning_FQ_list.append(u_fq_val)
        Unlearning_RU_list.append(u_ru_val)


    # Step 2: Comparison rules
    # A = Unlearning_FQ_list[i], B = Retrain_FQ_list[i] => FQ_list
    # A = Unlearning_RU_list[i], B = Retrain_RU_list[i] => RU_list

    def compare_vals(A_list, B_list):
        """
        Return the comparison list, with the same length as A_list and B_list.
        If A < B,        => A/B
        If B < A < 2B,   => 2 - A/B
        Else             => 0
        """
        output = []
        for A, B in zip(A_list, B_list):
            if B == 0:
                # If B=0, handle to avoid division by zero. Here we simply set it to 0.
                output.append(0)
            else:
                if A < B:
                    output.append(A / B)
                elif B < A < 2 * B:
                    output.append(2 - (A / B))
                else:
                    output.append(0)
        return output

    # Step 3: Construct final results FQ_list, RU_list
    FQ_list = compare_vals(Unlearning_FQ_list, Retrain_FQ_list)
    RU_list = compare_vals(Unlearning_RU_list, Retrain_RU_list)

    # Return the results
    return FQ_list, RU_list


In [ ]:
# Prepare the FQ and RU values for the retrained models
Llama_2_7b_chat_hf_Retrain_FQ_list = [26.00, 25.41, 24.83, 24.24, 23.67]
Llama_2_7b_chat_hf_Retrain_RU_list = [28.65, 29.43, 30.22, 31.00, 31.78]

Meta_Llama_3_8B_Retrain_FQ_list = [21.68, 20.98, 20.29, 19.59, 18.90]
Meta_Llama_3_8B_Retrain_RU_list = [25.71, 25.56, 25.40, 25.25, 25.10]

Meta_Llama_3_8B_Instruct_Retrain_FQ_list = [16.49, 16.17, 15.85, 15.53, 15.21]
Meta_Llama_3_8B_Instruct_Retrain_RU_list = [19.05, 19.46, 19.86, 20.27, 20.68]

Yi_6B_Retrain_FQ_list = [24.22, 23.32, 22.43, 21.53, 20.64]
Yi_6B_Retrain_RU_list = [28.96, 29.17, 29.37, 29.57, 29.78]


In [ ]:
# Ours
# Prepare results of our method
# Define the list of folder names
folders = [
    "Experiment_record/continuous/Llama-2-7b-chat-hf",
    "Experiment_record/continuous/Meta-Llama-3-8B",
    "Experiment_record/continuous/Meta-Llama-3-8B-Instruct",
    "Experiment_record/continuous/Yi-6B"
]

# Metrics fields to extract, the order matches the order of values in the final list
fields = [
    "forget_probability_score",
    "forget_rouge",
    "forget_acc",
    "retain_probability_score",
    "retain_rouge",
    "retain_acc"
]

# Structure to store results: {folder: {user_id: [6 average values]}}
continuous_results = {}

for folder in folders:
    # Match files with the pattern seed*_evaluations.json
    pattern = os.path.join(folder, "seed*_evaluations_all.json")
    file_list = glob.glob(pattern)
    
    # Use dictionary to group by user_id, storing cumulative sums and counts of metrics
    user_data = {}
    
    print(f"Processing folder: {folder}, found {len(file_list)} JSON files.")
    
    for file_path in file_list:
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                evaluations = json.load(f)
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")
            continue
        
        # Ensure the JSON file contains a list
        if not isinstance(evaluations, list):
            print(f"File {file_path} does not contain a list of evaluations.")
            continue
        
        # Iterate through each evaluation record
        for record in evaluations:
            user_id = record.get("user_id")
            if user_id is None:
                continue  # Skip if no user_id
            
            # Initialize cumulative dictionary if this user_id does not exist
            if user_id not in user_data:
                user_data[user_id] = {key: 0.0 for key in fields}
                user_data[user_id]["count"] = 0
            
            # Accumulate values for each metric
            for key in fields:
                if key in record:
                    user_data[user_id][key] += record[key]
                else:
                    print(f"Warning: {key} not found for user {user_id} in file {file_path}")
            user_data[user_id]["count"] += 1
    
    # Compute the average value for each user_id, build a dictionary where values are a list of 6 averages
    folder_result = {}
    for user_id, data in user_data.items():
        count = data.pop("count")
        if count > 0:
            avg_values = [data[key] / count for key in fields]
            folder_result[user_id] = avg_values
        else:
            print(f"No valid records for user {user_id} in folder {folder}.")
    
    continuous_results[folder] = folder_result

Llama_2_7b_chat_hf_list = []
Meta_Llama_3_8B_list = []
Meta_Llama_3_8B_Instruct_list = []
Yi_6B_list = []

# Output the results of each folder
for folder, folder_result in continuous_results.items():
    for user_id, avg_list in folder_result.items():
        if folder == f"{folders[0]}":
            Llama_2_7b_chat_hf_list.append(avg_list)
        elif folder == f"{folders[1]}":
            Meta_Llama_3_8B_list.append(avg_list)
        elif folder == f"{folders[2]}":
            Meta_Llama_3_8B_Instruct_list.append(avg_list)
        elif folder == f"{folders[3]}":
            Yi_6B_list.append(avg_list)

# Record results every 60 users
Llama_2_7b_chat_hf_list = Llama_2_7b_chat_hf_list
Meta_Llama_3_8B_list = Meta_Llama_3_8B_list
Meta_Llama_3_8B_Instruct_list = Meta_Llama_3_8B_Instruct_list
Yi_6B_list = Yi_6B_list


Processing folder: Experiment_record/continuous/Llama-2-7b-chat-hf, found 4 JSON files.


In [ ]:
# Calculate FQ and RU for our unlearning algorithm
Llama_2_7b_chat_hf_FQ_list, Llama_2_7b_chat_hf_RU_list = FQ_RU(
    Llama_2_7b_chat_hf_list, Llama_2_7b_chat_hf_Retrain_FQ_list, Llama_2_7b_chat_hf_Retrain_RU_list
)
Meta_Llama_3_8B_FQ_list, Meta_Llama_3_8B_RU_list = FQ_RU(
    Meta_Llama_3_8B_list, Meta_Llama_3_8B_Retrain_FQ_list, Meta_Llama_3_8B_Retrain_RU_list
)
Meta_Llama_3_8B_Instruct_FQ_list, Meta_Llama_3_8B_Instruct_RU_list = FQ_RU(
    Meta_Llama_3_8B_Instruct_list, Meta_Llama_3_8B_Instruct_Retrain_FQ_list, Meta_Llama_3_8B_Instruct_Retrain_RU_list
)
Yi_6B_FQ_list, Yi_6B_RU_list = FQ_RU(
    Yi_6B_list, Yi_6B_Retrain_FQ_list, Yi_6B_Retrain_RU_list
)

# Output results
print("Llama_2_7b_chat_hf_FQ_list:", Llama_2_7b_chat_hf_FQ_list)
print("Llama_2_7b_chat_hf_RU_list:", Llama_2_7b_chat_hf_RU_list)

print("Meta_Llama_3_8B_FQ_list:", Meta_Llama_3_8B_FQ_list)
print("Meta_Llama_3_8B_RU_list:", Meta_Llama_3_8B_RU_list)

print("Meta_Llama_3_8B_Instruct_FQ_list:", Meta_Llama_3_8B_Instruct_FQ_list)
print("Meta_Llama_3_8B_Instruct_RU_list:", Meta_Llama_3_8B_Instruct_RU_list)

print("Yi_6B_FQ_list:", Yi_6B_FQ_list)
print("Yi_6B_RU_list:", Yi_6B_RU_list)
